# Example of Funnel using Xcollection

In [129]:
from xcollection import Collection
import xcollection
import pydantic
import intake
import pandas as pd
from ncar_jobqueue import NCARCluster
from distributed import Client
import dask

In [34]:
catalog = intake.open_esm_datastore(
    'https://raw.githubusercontent.com/NCAR/cesm2-le-aws/main/data/aws-cesm2-le.json'
)
catalog

,unique
component,4
spatial_domain,3
frequency,3
path,284
experiment,2
forcing_variant,2
variable,57
long_name,57
start_time,4
end_time,6


In [58]:
cat = catalog.search(variable='SALT', component='ocn', frequency='monthly')

In [62]:
dsets = cat.to_dataset_dict(storage_options={'anon':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


In [30]:
cluster = NCARCluster()
cluster.scale(10)
client = Client(cluster)

In [31]:
client

Connection method: Cluster object,Cluster type: PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.3:40755,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [33]:
cluster

In [279]:
@pydantic.dataclasses.dataclass
class Funnel:
    dsets: xcollection.Collection
    
    def __post_init_post_parse__(self):
        self.keys = self.dsets.keys()
        self.base_keys = self.keys
        self.operator_keys = [None]
        self.base_token = dask.base.tokenize(self.dsets)
        
    def apply(self, operator, *operator_args, **operator_kwargs):
        operator_name = operator.__name__
        self.dsets = self.dsets.map(operator, *operator_kwargs, **operator_kwargs)
        self.operator_keys.append(operator_name)
        for key in self.dsets.keys():
            self.dsets[f'{key}.{operator_name}'] = self.dsets.pop(key)
        return self.dsets
    
    def df(self):
        dataframe = pd.DataFrame({'base_keys':self.base_keys,
                                  'operator_keys':self.operator_keys})
        self.database = dataframe
        return dataframe

In [280]:
collection = xcollection.Collection(dsets)

In [281]:
data_collection = Funnel(collection)

In [282]:
data_collection.df()

,base_keys,operator_keys
0,ocn.ssp370.monthly.cmip6,None


In [207]:
def func_a(ds):
    return ds.isel(member_id=0)

In [208]:
data_collection.apply(func_a)

ValueError: Dimensions {'member_id'} do not exist. Expected one or more of Frozen({'time': 1032, 'z_t': 60, 'nlat': 384, 'nlon': 320, 'd2': 2})

In [206]:
data_collection.df

,base_key,base_token,operator_keys
0,ocn.ssp370.monthly.cmip6,7ae281fbda764927c46e8a0e882bf505,None
